In [84]:
import math
from collections import Counter
import pickle
import re
# %pip install lyricsgenius
from lyricsgenius import Genius

with open('../lyrics.pkl', 'rb') as f:
    song_lyrics = pickle.load(f)


with open('../classifications.pkl', 'rb') as f1:
    classifications = pickle.load(f1)

with open('../big_songs.pkl', 'rb') as f2:
    big_songs = pickle.load(f2)

def tokenize(lyrics):
    return lyrics.lower().split()

k1 = 1.2
b = 0.2

In [85]:
# call the api to get the lyrics

genius = Genius('HhcPCGMwENzdfMQlauEV1Om0UormkWc6Wrwxri-LFTfBSNkzhwMSVUnR5tZG8eXW')
genius.remove_section_headers = True
genius.skip_non_songs = False
genius.excluded_terms = ["Remix", "Live"]

# song_name = "Happy"
# artist = "Pharrel Williams"
# song_name = "Firework"
# artist = "Katy Perry"
song_name = "I Hate Everything About You"
artist = "Three Days Grace"

song = genius.search_song(song_name, artist)
pos = song.lyrics.find(f'{song_name} Lyrics')
query = song.lyrics[pos:]



Searching for "Venom" by Eminem...
Done.


In [86]:
# then remove  capital letters and non alphanumeric characters
query = query.lower()
query = ''.join([char if char.isalpha() or char.isspace() else '' for char in query.replace('\n', ' ')])
# it might seem crazy what im bout to say
query



'venom lyrics i got a song filled with shit for the strongwilled when the world gives you a raw deal sets you off til you scream piss off screw you when it talks to you like you dont belong or tells you youre in the wrong field when somethings in your mitochondrial cause it latched on to you like  knock knock let the devil in manevolent as ive ever been head is spinnin this medicines screamin llllet us in llllike a salad bowl edgar allan poe bedridden shoulda been dead a long time ago liquid tylenol gelatins think my skeletons meltin wicked i get all high when i think ive smelled the scent of elephant manurehell i meant kahlúa screw it to hell with it i went through hell with accelerants and blew up mymymyself again volkswagen tailspin bucket matches my pale skin mayo and went from hellmanns and being rail thin filetofish scribble jam rap olympics  freaknik how can i be down me and bizarre in florida proofs room slept on the floor of da motel then dr dre said hell yeah and i got his st

In [87]:
# tokenize all documents
tokenized_songs = {}
for category, all_lyrics in big_songs.items():
    tokenized_songs[category] = tokenize(all_lyrics)

# calculate document lengths
doc_lengths = {}
for  category, all_lyrics in tokenized_songs.items():
    doc_lengths[category] = len(all_lyrics)

# avg doc length
avg_doc_length = sum(doc_lengths.values()) / len(doc_lengths)


In [88]:
# calculate doc freqs
doc_freqs = {}
for doc in tokenized_songs.values():
    unique_terms = set(doc)
    for term in unique_terms:
        doc_freqs[term] = doc_freqs.get(term, 0) + 1

In [89]:
# bm25 algorithm
def bm25(query, document, doc_length):
    score = 0.0
    query_terms = tokenize(query)
    term_freqs = Counter(document)
    
    for term in query_terms:
        if term in doc_freqs:  # term must appear in the corpus
            # Calculate BM25 components
            doc_freq = doc_freqs[term]
            idf = math.log((len(big_songs) - doc_freq + 0.5) / (doc_freq + 0.5) + 1)
            term_freq = term_freqs[term]
            term_score = idf * (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * doc_length / avg_doc_length))
            score += term_score
    return score

In [90]:
scores = []
for doc_id, doc in tokenized_songs.items():
    score = bm25(query, doc, doc_lengths[doc_id])
    scores.append((doc_id, score))

# Sort documents by score in descending order
ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)

# Display the ranking results
print("Document Rankings:")
for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
    print(f"Rank {rank}: Document {doc_id} with score {score:.4f}")

Document Rankings:
Rank 1: Document uplifting with score 286.2855
Rank 2: Document dark with score 259.6492
Rank 3: Document energetic with score 237.5996
Rank 4: Document dynamic with score 207.2955
Rank 5: Document anger with score 199.8152
Rank 6: Document fierce with score 193.4736
Rank 7: Document cynical with score 46.6294
